In [1]:
from locator import *
from DB_work import DB_ORM
import os
from random import choice
from numpy.random import uniform as random_

In [2]:
cargo_table = 'CARGOS_STORED'

cargo_id_sequence = '0123456789ABCDEF'
cargo_id_length = 16

In [3]:
def calculator(length = cargo_id_length, digits = cargo_id_sequence):
    res = str(length ** len(digits))
    return(' '.join([''.join([a for a in res[::-1][x - 3 : x]]) for x in range(3, length + 7, 3)])[::-1])

In [4]:
def generate_hex_sequence(length = cargo_id_length, seq_len = 4, digits = cargo_id_sequence):
    return('-'.join( [''.join( [choice(digits) for i in range(seq_len) ] ) for j in range(round(length / seq_len)) ] ))

In [5]:
print(f"""Выбранный метод учета идентификаторов грузов позволяет иметь {calculator()} грузов с уникальным идентификатором""")

Выбранный метод учета идентификаторов грузов позволяет иметь 18 446 744 073 709 551 616 грузов с уникальным идентификатором


In [6]:
def random(low, high, size, is_round = False):
    return([round(i, 2) if not is_round else round(i) for i in random_(low, high, size)])

In [7]:
DB_object = DB_ORM()

In [11]:
cargos_packages = [
    {'name' : 'Малый', 'sizes' : '30x40x30', 'max_weight' : 10, 'fragile' : True}, 
    {'name' : 'Средний', 'sizes' : '60x60x50', 'max_weight' : 40, 'fragile' : True}, 
    {'name' : 'Большой', 'sizes' : '100x70x70', 'max_weight' : 70, 'fragile' : False}
    ]

In [8]:
cargos_data_tests = [{
    'Вес' : random(0.5, 10, 1)[0], 
    "Габариты" : f"{random(10, 100, 1, True)[0]}x{random(10, 70, 1, True)[0]}x{random(10, 70, 1, True)[0]}", 
    'Хрупкое' : random(0, 1, 1, True)[0]
    } for i in range(10)]

In [9]:
cargos_data_tests

[{'Вес': 7.99, 'Габариты': '16x52x64', 'Хрупкое': 1},
 {'Вес': 2.19, 'Габариты': '58x43x46', 'Хрупкое': 0},
 {'Вес': 0.58, 'Габариты': '16x62x50', 'Хрупкое': 0},
 {'Вес': 6.74, 'Габариты': '98x28x47', 'Хрупкое': 1},
 {'Вес': 5.16, 'Габариты': '30x11x32', 'Хрупкое': 0},
 {'Вес': 8.06, 'Габариты': '90x38x59', 'Хрупкое': 0},
 {'Вес': 7.35, 'Габариты': '74x68x15', 'Хрупкое': 1},
 {'Вес': 8.72, 'Габариты': '52x50x47', 'Хрупкое': 1},
 {'Вес': 4.35, 'Габариты': '33x64x67', 'Хрупкое': 1},
 {'Вес': 2.83, 'Габариты': '47x66x12', 'Хрупкое': 1}]

---

Занесение в БД грузов

- Все большие грузы с пометкой "Хрупкое" не могут быть размещены на складе. Необходимо добавление новых контейнеров

In [14]:
unplaced_cargos = []

for cargo_number, cargo in enumerate(cargos_data_tests):
    for cargo_package_info in cargos_packages:
        is_ok = [False, False, False]

        if cargo['Хрупкое']:
            if cargo_package_info['fragile']:
                is_ok[0] = True
        
        else:
            is_ok[0] = True


        if cargo['Вес'] <= cargo_package_info['max_weight']:
            is_ok[1] = True
        

        cargo_sizes = cargo['Габариты'].split('x')
        cargo_package_sizes = cargo_package_info['sizes'].split('x')


        if int(cargo_sizes[0]) <= int(cargo_package_sizes[0]) and int(cargo_sizes[1]) <= int(cargo_package_sizes[1]) and int(cargo_sizes[2]) <= int(cargo_package_sizes[2]):
            is_ok[2] = True
        
        if is_ok.count(True) == 3:
            break
    
    

    if is_ok.count(True) == 3:
        zone = 'CARGOS_ZONE_' + {'Малый' : 'SMALL', 'Средний' : 'MIDDLE', 'Большой' : 'BIG'}[cargo_package_info['name']]

        cargo_id_prep = generate_hex_sequence()


        cargo_id_exists = True if len(DB_object.execute(f"""
                                                SELECT * FROM {cargo_table} WHERE cargo_id = '{cargo_id_prep}'
                                            """, is_change=False)) else False
        
        while cargo_id_exists:
            cargo_id_prep = generate_hex_sequence()
            cargo_id_exists = True if len(DB_object.execute(f"""
                                            SELECT * FROM {cargo_table} WHERE cargo_id = '{cargo_id_prep}'
                                        """, is_change=False)) else False

        # print(cargo_id_prep, cargo_id_exists)
        # print(cargo, cargo_package_info)
        # print(zone, '\n')

        DB_object.execute(f"""UPDATE {zone} SET cargo_id = '{cargo_id_prep}' WHERE id = (SELECT id from {zone} WHERE cargo_id is NULL LIMIT 1);""", is_change=True)

        DB_object.execute(f"""
                            INSERT INTO {cargo_table} (cargo_id, weight, long_width_height, fragile, zone) VALUES ('{cargo_id_prep}', {cargo['Вес']}, '{cargo['Габариты']}', {cargo['Хрупкое']}, '{zone}');
                        
                            """, is_change=True)
    else:
        unplaced_cargos.append(cargo)

new_row_sym = '\n'
print(f"""
      Успешно размещено {len([str(i) for i in cargos_data_tests if i not in unplaced_cargos])} груз(-а, -ов):
        {new_row_sym.join([str(i) for i in cargos_data_tests if i not in unplaced_cargos])}

      Не удалось разместить {len(unplaced_cargos)} груз(-а, -ов):
        {new_row_sym.join([str(i) for i in unplaced_cargos])}
      """)


      Успешно размещено 5 груз(-а, -ов):
        {'Вес': 2.19, 'Габариты': '58x43x46', 'Хрупкое': 0}
{'Вес': 0.58, 'Габариты': '16x62x50', 'Хрупкое': 0}
{'Вес': 5.16, 'Габариты': '30x11x32', 'Хрупкое': 0}
{'Вес': 8.06, 'Габариты': '90x38x59', 'Хрупкое': 0}
{'Вес': 8.72, 'Габариты': '52x50x47', 'Хрупкое': 1}

      Не удалось разместить 5 груз(-а, -ов):
        {'Вес': 7.99, 'Габариты': '16x52x64', 'Хрупкое': 1}
{'Вес': 6.74, 'Габариты': '98x28x47', 'Хрупкое': 1}
{'Вес': 7.35, 'Габариты': '74x68x15', 'Хрупкое': 1}
{'Вес': 4.35, 'Габариты': '33x64x67', 'Хрупкое': 1}
{'Вес': 2.83, 'Габариты': '47x66x12', 'Хрупкое': 1}
      


---

Получение из БД

In [26]:
cargo_id_find = 'D961-CA99-BF73-FB44'

cargo_data = DB_object.execute(f"""SELECT * FROM CARGOS_STORED WHERE cargo_id = '{cargo_id_find}'""", is_change=False)

is_cargo_stored = True if len(cargo_data) else False

if not is_cargo_stored:
    print('Груза с таким ID нет на складе')

else:
    cargo_data = cargo_data[0]
    columns = DB_object.columns_names(cargo_table)
    cargo_data = {column : value for column, value in zip(columns, cargo_data)}

    cell_number = DB_object.execute(f"""SELECT * FROM {cargo_data['zone']} WHERE cargo_id = '{cargo_id_find}'""", is_change=False)[0][0]
    print(f"""Груз {cargo_id_find} 
    {cargo_data} 
    расположен в зоне {cargo_data['zone']} в ячейке # {cell_number}""")

Груз D961-CA99-BF73-FB44 
    {'id': 35, 'cargo_id': 'D961-CA99-BF73-FB44', 'weight': 7.68, 'long_width_height': '72x32x60', 'fragile': 0, 'zone': 'CARGOS_ZONE_BIG'} 
    расположен в зоне CARGOS_ZONE_BIG в ячейке # 23


---

Имитация выдачи груза

In [28]:
cargo_id_out = 'D961-CA99-BF73-FB44'

cargo_data = DB_object.execute(f"""SELECT * FROM CARGOS_STORED WHERE cargo_id = '{cargo_id_find}'""", is_change=False)

is_cargo_stored = True if len(cargo_data) else False

if not is_cargo_stored:
    print('Груза с таким ID нет на складе')

else:
    cargo_data = cargo_data[0]
    columns = DB_object.columns_names(cargo_table)
    cargo_data = {column : value for column, value in zip(columns, cargo_data)}

    cell_number = DB_object.execute(f"""SELECT * FROM {cargo_data['zone']} WHERE cargo_id = '{cargo_id_find}'""", is_change=False)[0][0]

    DB_object.execute(f"""UPDATE {cargo_data['zone']} SET cargo_id = NULL WHERE id = {cell_number}""", is_change=True)
    DB_object.execute(f"""DELETE FROM {cargo_table} WHERE cargo_id = '{cargo_id_find}'""", is_change=True)

    print(f"""Груз {cargo_id_find} 
    {cargo_data} 
    расположенный в зоне {cargo_data['zone']} в ячейке # {cell_number} успешно выдан""")



Груз D961-CA99-BF73-FB44 
    {'id': 35, 'cargo_id': 'D961-CA99-BF73-FB44', 'weight': 7.68, 'long_width_height': '72x32x60', 'fragile': 0, 'zone': 'CARGOS_ZONE_BIG'} 
    расположенный в зоне CARGOS_ZONE_BIG в ячейке # 23 успешно выдан


---

In [294]:
DB_object.execute('DROP TABLE CARGOS_ZONE_SMALL', is_change=True)

True

In [295]:
DB_object.execute('DROP TABLE CARGOS_ZONE_MIDDLE', is_change=True)

True

In [296]:
DB_object.execute('DROP TABLE CARGOS_ZONE_BIG', is_change=True)

True

In [297]:
# DB_object.execute('DROP TABLE CARGOS_STORED', is_change=True)

In [298]:
DB_object.execute(f'DROP TABLE {cargo_table}', is_change=True)

True

In [299]:
DB_object.execute(f"""
                  CREATE TABLE IF NOT EXISTS CARGOS_ZONE_SMALL(
                    id INTEGER PRIMARY KEY,
                    cargo_id INTEGER UNIQUE
                  )
                  """, is_change=True)


DB_object.execute(f"""
                  CREATE TABLE IF NOT EXISTS CARGOS_ZONE_MIDDLE(
                    id INTEGER PRIMARY KEY,
                    cargo_id INTEGER UNIQUE
                  )
                  """, is_change=True)


DB_object.execute(f"""
                  CREATE TABLE IF NOT EXISTS CARGOS_ZONE_BIG(
                    id INTEGER PRIMARY KEY,
                    cargo_id INTEGER UNIQUE
                  )
                  """, is_change=True)

DB_object.execute(f"""
                  CREATE TABLE IF NOT EXISTS {cargo_table} (
                        id INTEGER UNIQUE PRIMARY KEY,
                        cargo_id TEXT UNIQUE,
                        weight REAL,
                        long_width_height TEXT,
                        fragile INTEGER,
                        zone TEXT
                    );
                  """, is_change=True)

True

In [300]:
for name in ('SMALL', 'MIDDLE', 'BIG'):
    for i in range(100):
        DB_object.execute(f"""
                      INSERT INTO CARGOS_ZONE_{name} (id) VALUES({i})
                  """, is_change=True)